<a href="https://colab.research.google.com/github/VittorioBartolomeoSecondin/DVIS-CAValli_Team/blob/main/Maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries and connecting to Google Drive

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import re
import json
import csv
import requests
import os
import zipfile
from google.colab import files
from google.colab import drive
drive.mount('/content/drive') # turned out to be a good workaround to load a huge amount of data and keep it available

Mounted at /content/drive


# Load tree_dataset

In [2]:
tree_dataset = pd.read_csv("/content/drive/MyDrive/tree_dataset/final_dataset/tree_dataset.csv", low_memory=False)

# Computing tree abundance for each state

In [3]:
def create_abundance_csv(csv_name, header, name, abundance, postal):
  with open(csv_name, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for idx in range(len(state)):
      row = [name[idx], abundance[idx], postal.get(name[idx])]
      writer.writerow(row)

In [4]:
postal_codes = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "District of Columbia": "DC",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Puerto Rico": "PR",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Virgin Islands": "VI",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}

In [5]:
csv_name = "state_abundance_postal.csv"
header = ['state', 'abundance', 'postal']
state = list(tree_dataset['state'].value_counts().index)
abundance = list(tree_dataset['state'].value_counts().values)
create_abundance_csv(csv_name, header, state, abundance, postal_codes)

# Choropleth: JSON file

In [6]:
abundance_df = pd.read_csv("/content/state_abundance_postal.csv", low_memory=False)
abundance_dict = {}

for idx in range(abundance_df.shape[0]):
  name = list(abundance_df.iloc[idx])[0]
  abundance = list(abundance_df.iloc[idx])[1]
  postal = list(abundance_df.iloc[idx])[2]
  abundance_dict[postal] = [name, abundance]

for key, value in postal_codes.items():
  if abundance_dict.get(value) is None:
    name = key
    abundance = 0
    abundance_dict[value] = [name, abundance]
  else:
    name = abundance_dict.get(value)[0]
    abundance = abundance_dict.get(value)[1]

In [7]:
json_path = "/content/drive/MyDrive/tree_dataset/final_dataset/us-states.json"
with open(json_path, 'r') as file:
    json_file = json.load(file)

In [8]:
areas_dict = {
    'Alabama': 135767,
    'Alaska': 1717855,
    'Arizona': 295234,
    'Arkansas': 137733,
    'California': 423968,
    'Colorado': 269601,
    'Connecticut': 12534,
    'Delaware': 5047,
    'District of Columbia': 177,
    'Florida': 170304,
    'Georgia': 148958,
    'Hawaii': 16637,
    'Idaho': 216443,
    'Illinois': 144774,
    'Indiana': 92789,
    'Iowa': 145746,
    'Kansas': 211754,
    'Kentucky': 102908,
    'Louisiana': 135659,
    'Maine': 91646,
    'Maryland': 25313,
    'Massachusetts': 20203,
    'Michigan': 250488,
    'Minnesota': 225163,
    'Mississippi': 120383,
    'Missouri': 180560,
    'Montana': 380832,
    'Nebraska': 199097,
    'Nevada': 286380,
    'New Hampshire': 23186,
    'New Jersey': 19050,
    'New Mexico': 314917,
    'New York': 141297,
    'North Carolina': 139390,
    'North Dakota': 178712,
    'Ohio': 116096,
    'Oklahoma': 177846,
    'Oregon': 248608,
    'Pennsylvania': 119283,
    'Rhode Island': 2678,
    'South Carolina': 82931,
    'South Dakota': 199729,
    'Tennessee': 109153,
    'Texas': 695662,
    'Utah': 219882,
    'Vermont': 24906,
    'Virginia': 110787,
    'Washington': 184666,
    'West Virginia': 62259,
    'Wisconsin': 140268,
    'Wyoming': 253335
}

In [9]:
for idx in range(len(json_file['objects']['states'].get('geometries'))):
  properties = json_file['objects']['states'].get('geometries')[idx]['properties']
  postal = properties['postal']
  try:
    name, abundance = abundance_dict[postal]
    area = areas_dict[name]
    properties['name'] = name
    properties['abundance'] = str(abundance)
    properties['area'] = area
    properties['density'] = round(abundance / area, 5)
    properties['density_1000'] = round(abundance / area * 1000, 2)
    print(properties)
  except:
    pass

{'postal': 'AK', 'name': 'Alaska', 'abundance': '0', 'area': 1717855, 'density': 0.0, 'density_1000': 0.0}
{'postal': 'AL', 'name': 'Alabama', 'abundance': '0', 'area': 135767, 'density': 0.0, 'density_1000': 0.0}
{'postal': 'AR', 'name': 'Arkansas', 'abundance': '0', 'area': 137733, 'density': 0.0, 'density_1000': 0.0}
{'postal': 'AZ', 'name': 'Arizona', 'abundance': '214', 'area': 295234, 'density': 0.00072, 'density_1000': 0.72}
{'postal': 'CA', 'name': 'California', 'abundance': '1806085', 'area': 423968, 'density': 4.25996, 'density_1000': 4259.96}
{'postal': 'CO', 'name': 'Colorado', 'abundance': '354878', 'area': 269601, 'density': 1.31631, 'density_1000': 1316.31}
{'postal': 'CT', 'name': 'Connecticut', 'abundance': '0', 'area': 12534, 'density': 0.0, 'density_1000': 0.0}
{'postal': 'DC', 'name': 'District of Columbia', 'abundance': '175847', 'area': 177, 'density': 993.48588, 'density_1000': 993485.88}
{'postal': 'DE', 'name': 'Delaware', 'abundance': '0', 'area': 5047, 'densi

In [10]:
out_path = "/content/choropleth.json"
with open(out_path, 'w') as file:
    json.dump(json_file, file)

# Dot map: CSV file

We drop data with no longitude/latitude.

In [11]:
tree_dataset.dropna(subset=['longitude_coordinate', 'latitude_coordinate'], how='any', inplace=True)
print('The total number of remaining trees is:', tree_dataset.shape[0])

The total number of remaining trees is: 4556965


We round longitude/latitude to the first decimal number.

In [12]:
dotmap_full = tree_dataset[['state', 'longitude_coordinate', 'latitude_coordinate', 'scientific_name', 'common_name', 'tree_genus', 'tree_family', 'tree_order']]
dotmap_full['longitude_coordinate_rounded'] = round(dotmap_full['longitude_coordinate'], 1)
dotmap_full['latitude_coordinate_rounded'] = round(dotmap_full['latitude_coordinate'], 1)

<ipython-input-12-d0f0656b8c31>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dotmap_full['longitude_coordinate_rounded'] = round(dotmap_full['longitude_coordinate'], 1)
<ipython-input-12-d0f0656b8c31>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dotmap_full['latitude_coordinate_rounded'] = round(dotmap_full['latitude_coordinate'], 1)


We define the boundaries of each state to drop data that do not fall within.

In [13]:
# Assuming state_border_coordinates is a dictionary with state names as keys and border coordinates as values
state_border_coordinates = {
    'California': {'min_longitude': -124.7, 'max_longitude': -113.9, 'min_latitude': 32.3, 'max_latitude': 42.2},
    'Colorado': {'min_longitude': -109.3, 'max_longitude': -101.8, 'min_latitude': 36.8, 'max_latitude': 41.2},
    'District of Columbia': {'min_longitude': -77.13, 'max_longitude': -76.9, 'min_latitude': 38.79, 'max_latitude': 39},
    'Florida': {'min_longitude': -87.85, 'max_longitude': -79.8, 'min_latitude': 24.3, 'max_latitude': 31.2},
    'Georgia': {'min_longitude': -85.8, 'max_longitude': -80.6, 'min_latitude': 30.2, 'max_latitude': 35.2},
    'Hawaii': {'min_longitude': -160.7, 'max_longitude': -154.6, 'min_latitude': 18.7, 'max_latitude': 22.4},
    'Iowa': {'min_longitude': -96.8, 'max_longitude': -89.9, 'min_latitude': 40.2, 'max_latitude': 43.7},
    'Kansas': {'min_longitude': -102.3, 'max_longitude': -94.4, 'min_latitude': 36.8, 'max_latitude': 40.2},
    'Kentucky': {'min_longitude': -89.8, 'max_longitude': -81.8, 'min_latitude': 35.8, 'max_latitude': 39.8},
    'Maryland': {'min_longitude': -79.7, 'max_longitude': -74.8, 'min_latitude': 37.6, 'max_latitude': 40},
    'Michigan': {'min_longitude': -90.6, 'max_longitude': -81.9, 'min_latitude': 41.5, 'max_latitude': 48.5},
    'Minnesota': {'min_longitude': -97.4, 'max_longitude': -89.3, 'min_latitude': 43.3, 'max_latitude': 49.6},
    'Missouri': {'min_longitude': -96, 'max_longitude': -88.9, 'min_latitude': 35.8, 'max_latitude': 40.8},
    'Nevada': {'min_longitude': -120.1, 'max_longitude': -113.9, 'min_latitude': 34.9, 'max_latitude': 42.2},
    'New Mexico': {'min_longitude': -109.3, 'max_longitude': -102.8, 'min_latitude': 31.1, 'max_latitude': 37.2},
    'New York': {'min_longitude': -80, 'max_longitude': -71.7, 'min_latitude': 40.3, 'max_latitude': 45.3},
    'North Carolina': {'min_longitude': -84.5, 'max_longitude': -75.2, 'min_latitude': 32.8, 'max_latitude': 37.5},
    'Ohio': {'min_longitude': -85, 'max_longitude': -80.3, 'min_latitude': 38.2, 'max_latitude': 42.5},
    'Oklahoma': {'min_longitude': -103.2, 'max_longitude': -94.2, 'min_latitude': 33.4, 'max_latitude': 37.2},
    'Oregon': {'min_longitude': -124.9, 'max_longitude': -116.3, 'min_latitude': 41.8, 'max_latitude': 46.5},
    'Pennsylvania': {'min_longitude': -80.7, 'max_longitude': -74.5, 'min_latitude': 39.5, 'max_latitude': 42.7},
    'Rhode Island': {'min_longitude': -72.1, 'max_longitude': -70.9, 'min_latitude': 41, 'max_latitude': 42.2},
    'South Dakota': {'min_longitude': -104.3, 'max_longitude': -96.2, 'min_latitude': 42.3, 'max_latitude': 46.1},
    'Tennessee': {'min_longitude': -90.5, 'max_longitude': -81.4, 'min_latitude': 33.6, 'max_latitude': 38.1},
    'Texas': {'min_longitude': -106.9, 'max_longitude': -93.2, 'min_latitude': 25.5, 'max_latitude': 36.8},
    'Washington': {'min_longitude': -125, 'max_longitude': -116.7, 'min_latitude': 45.3, 'max_latitude': 49.2},
    'Wisconsin': {'min_longitude': -93.1, 'max_longitude': -86, 'min_latitude': 42.3, 'max_latitude': 47.5}
}

## Dot map: 1 class

In [14]:
dotmap = dotmap_full[['state', 'longitude_coordinate_rounded', 'latitude_coordinate_rounded']
                      ].value_counts().reset_index().sort_values(
                          by = ['state', 'longitude_coordinate_rounded', 'latitude_coordinate_rounded'])
dotmap.columns = ['state', 'longitude', 'latitude', 'count']

We drop data that do not fall within the boundaries of each state

In [15]:
rows_to_drop = []
for index, row in dotmap.iterrows():
    state = row['state']
    longitude = row['longitude']
    latitude = row['latitude']

    state_coords = state_border_coordinates.get(state)
    if not (state_coords['min_longitude'] <= longitude <= state_coords['max_longitude'] \
            and state_coords['min_latitude'] <= latitude <= state_coords['max_latitude']):
        rows_to_drop.append(index)

dotmap.drop(rows_to_drop, inplace=True) # keeping only trees within the boundaries of the associated state

In [16]:
print('The number of observations is:', dotmap.shape[0])
print('The total number of trees considered is:', dotmap['count'].sum())

The number of observations is: 596
The total number of trees considered is: 4534908


In [17]:
dotmap.to_csv('dotmap.csv', index=False)

## Dot map: 10 classes + *Others*

In [18]:
dotmap2 = dotmap_full[['state', 'longitude_coordinate_rounded', 'latitude_coordinate_rounded', 'scientific_name']
                      ].value_counts().reset_index().sort_values(
                          by = ['state', 'longitude_coordinate_rounded', 'latitude_coordinate_rounded', 'scientific_name'])
dotmap2.columns = ['state', 'longitude', 'latitude', 'scientific_name', 'count']

We drop data that do not fall within the boundaries of each state

In [19]:
rows_to_drop = []
for index, row in dotmap2.iterrows():
    state = row['state']
    longitude = row['longitude']
    latitude = row['latitude']

    state_coords = state_border_coordinates.get(state)
    if not (state_coords['min_longitude'] <= longitude <= state_coords['max_longitude'] \
            and state_coords['min_latitude'] <= latitude <= state_coords['max_latitude']):
        rows_to_drop.append(index)

dotmap2.drop(rows_to_drop, inplace=True) # keeping only trees within the boundaries of the associated state

We retrieve the top 10 species (among those trees with correct longitude/latitude values).

In [20]:
dotmap2['total_count'] = dotmap2.groupby('scientific_name')['count'].transform('sum')
dotmap2_sorted = dotmap2.sort_values(by = ['total_count', 'scientific_name'], ascending = [False, True])
dotmap2_top = dotmap2_sorted[['scientific_name', 'total_count']].drop_duplicates().head(10)
top_species = list(dotmap2_top['scientific_name'])
print(top_species)

['Gleditsia triacanthos', 'Platanus acerifolia', 'Acer platanoides', 'Lagerstroemia indica', 'Pyrus calleryana', 'Fraxinus pennsylvanica', 'Acer rubrum', 'Liquidambar styraciflua', 'Tilia cordata', 'Prunus']


We put the remaining ones in the *others* class.

In [21]:
dotmap2['scientific_name'] = dotmap2.apply(lambda row: 'Others' if row['scientific_name'] not in top_species else row['scientific_name'], axis=1)

We aggregate in one observation all the trees in *Others* that were distinct before.

In [22]:
dotmap2.drop('total_count', axis=1, inplace=True)
dotmap2['count'] = dotmap2.groupby(['state', 'longitude', 'latitude', 'scientific_name'])['count'].transform('sum')
dotmap2.drop_duplicates(inplace=True)

In [23]:
print('The number of observations is:', dotmap2.shape[0])
print('The total number of trees considered is:', dotmap2['count'].sum())

The number of observations is: 3468
The total number of trees considered is: 4534908


In [24]:
dotmap2.to_csv('dotmap2.csv', index=False)